In [1]:
!pip install facenet_pytorch

In [2]:
!pip install matplotlib

In [24]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn

import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob
import os
# from tensorflow.keras.models import load_model

# from tensorflow.keras.models import Model
import os
import warnings


from torch.utils.data import random_split
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim



In [25]:

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.gender_mapping = {'male': 0, 'female': 1}
        self.ethnicity_mapping = {'white': 0, 'black': 1, 'asian': 2, 'hispanic': 3}
        self.data['nameNum'] = self.data['name'].astype('category').cat.codes
        self.data['nameNum'] = self.data['nameNum'].astype(int)


    def __len__(self):
        return len(self.data)


    
    def getAgeLabel(self,value1):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]
    
    # Check if both values fall into the same class range
        if(class_ranges[0][0]<=value1 and value1<class_ranges[0][1]):
            return 0
        elif(class_ranges[1][0]<=value1 and value1<class_ranges[1][1]):
            return 1
        elif(class_ranges[2][0]<=value1 and value1<class_ranges[2][1]):
            return 2
        elif(class_ranges[3][0]<=value1 and value1<class_ranges[3][1]):
            return 3
        else:
            return 0
   

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = '/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/Data/CELEBTEST/'+row['name']+'/' + row['filename']  # Assuming images are in a folder named 'images'
        
        try:
            image = Image.open(image_path)
        except Exception as e:
            # Handle the error, for example, you can return a placeholder image
            print("here")
            #self.__getitem__(idx + 1)
            #image = Image.new('RGB', (224, 224))  # Create a blank image
        
        image = Image.open(image_path)
        age = row['age']
        
        if(row['age']<=0):
            age=35
        label = {
            'age': self.getAgeLabel(age),
            'gender': self.gender_mapping.get(row['gender'], 0),  # -1 for unknown
            'ethnicity': self.ethnicity_mapping.get(row['ethnicity'], 0),
            'age1':age,
            'name': row['nameNum']
        
        }
        #print(row['name'])
        if self.transform:
            image = self.transform(image)

        return image, label


In [26]:
# csv_file = '/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv'  # Replace with the actual path to your CSV file
# df = pd.read_csv(csv_file)

# # Create a list to store the indices of rows with missing files
# rows_to_remove = []
# count=0
# # Iterate through the DataFrame and check if the files exist
# for index, row in df.iterrows():
#     image_path = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename'] 
#     if not os.path.exists(image_path):
#         rows_to_remove.append(index)
#         count=count+1
# df = df.drop(rows_to_remove)
# df.to_csv(csv_file, index=False)
# print(count)

In [27]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the image to the desired size
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


In [39]:
# trainSet = CustomDataset('/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/celebSetIdentity.csv', transform=transform, get_pairs=True, num_pairs=25)
# trainloader = DataLoader(trainSet, batch_size=16, shuffle=True, num_workers = 2)

# testSet = CustomDataset('/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/celebSetIdentityTest.csv', transform=transform, get_pairs=False)
# testloader = DataLoader(testSet, batch_size=16, shuffle=False)

trainSet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv', transform=transform)
trainloader = DataLoader(trainSet, batch_size=1, shuffle=False, num_workers=2)

testSet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/final_clebSET_test.csv', transform=transform)
testloader = DataLoader(testSet, batch_size=1, shuffle=False, num_workers=2)


In [40]:
def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

In [41]:
# tempIter = iter(testloader)
# images,gen_pairs, imp_pairs = next(tempIter)
# # imshow(images[0])
# # print(labels['age'][0],labels['gender'][0],labels['ethnicity'][0])
# # print(image_pairs)
# imshow(images[0])
# imshow(gen_pairs[0][0])
# imshow(imp_pairs[0][0])

In [42]:
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=FutureWarning) 
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print('Running on device: {}'.format(device))

Running on device: cuda:1


In [43]:
trainSet.data['age'].min()

-9

In [44]:
def get_normalized_age_value(original_age_value):
    return (original_age_value - trainSet.data['age'].min())/(trainSet.data['age'].max() - trainSet.data['age'].min())


def get_original_age_value(normalized_age_value):
    return normalized_age_value * (trainSet.data['age'].max()  - trainSet.data['age'].min()) + trainSet.data['age'].min()

In [45]:
file_path = '/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Code/MRL+PP+FHE/MRL+PP+FHE_AdaFace_Celebset.txt'
# Open the file and read its lines into a list of strings without newline characters
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file]

# Display the list of strings
print(len(lines))
print(lines[0])


72489
...


In [46]:
print(len(lines[0]))
print(len(lines[1]))

30824
30828


In [47]:
max_length = max(lines, key=len)
print(len(max_length))

30843


In [48]:
padded_lines = [s.ljust(len(max_length), '0') for s in lines]

In [50]:
def ageLabel(age):

    # Define class ranges
    class_ranges = [(15, 22),(22,40),(40,60),(60,80)]

# Check if both values fall into the same class range
    if(class_ranges[0][0] <= age and age < class_ranges[0][1]):
        return 0
    elif(class_ranges[1][0] <= age and age < class_ranges[1][1]):
        return 1
    elif(class_ranges[2][0] <= age and age < class_ranges[2][1]):
        return 2
    elif(class_ranges[3][0] <= age and age < class_ranges[3][1]):
        return 3
    else:
        return 0
    
def genderMapping(gender):
    if gender == 'male':
        return 0
    elif gender == 'female':
        return 1
    return -1

def ethMapping(eth):
    if eth == 'white':
        return 0
    elif eth == 'black':
        return 1
    elif eth == 'asian':
        return 2
    elif eth == 'hispanic':
        return 3
    return -1


In [51]:
data = pd.read_csv('/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv')
data['nameNum'] = data['name'].astype('category').cat.codes
data['nameNum'] = data['nameNum'].astype(int)



In [52]:

age_list = []
gender_list = []
ethn_list =[]
id_list = []


for i in range(len(data)):
    row = data.iloc[i]
    id_list.append(row['nameNum'])
    gender_list.append(genderMapping(row['gender']))
    ethn_list.append(ethMapping(row['ethnicity']))
    if row['age'] <= 0:
        age_list.append(ageLabel(35))
    else:
        age_list.append(row['age'])

In [59]:
test_data = pd.read_csv('/home/csgrad/byalavar/FHE/celebSet/final_clebSET_test.csv')
test_data['nameNum'] = test_data['name'].astype('category').cat.codes
test_data['nameNum'] = test_data['nameNum'].astype(int)

for i in range(len(test_data)):
    row = test_data.iloc[i]
    id_list.append(row['nameNum'])
    gender_list.append(genderMapping(row['gender']))
    ethn_list.append(ethMapping(row['ethnicity']))
    if row['age'] <= 0:
        age_list.append(ageLabel(35))
    else:
        age_list.append(row['age'])

In [60]:
print("Age length = ", len(age_list))
print("Eth length = ", len(ethn_list))
print("Gender length = ", len(gender_list))
print("Id length = ", len(id_list))

Age length =  72489
Eth length =  72489
Gender length =  72489
Id length =  72489


In [55]:
!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#Age SVM

In [61]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, age_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.06118085253138364


#Gender SVM

In [62]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, gender_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5222099599944819


#Id SVM

In [63]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, id_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.04035039315767692


#Eth SVM

In [64]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, ethn_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8012139605462822
